In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install smogn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import pandas as pd
import smogn
#dataset=pd.read_csv("/content/drive/MyDrive/Iqram Sir/KOA_EMG_WOMAC__2022.08.29_Final_W-tempo-spatial.csv")
d=pd.read_csv("/content/drive/MyDrive/Iqram Sir/KOA_EMG_WOMAC__2022.08.31_Final_Original.csv")

dataset = smogn.smoter(
    
    data = d, 
    y = "WOMAC_total"
)

r_index: 100%|##########| 2/2 [00:00<00:00,  3.33it/s]


In [5]:
dataset.head(10)

,Patient ID,Data Type,Bone-Attribution,WOMAC_Pain,WOMAC_Pain Grade,WOMAC_Stiffness,WOMAC_Stiffness Grade,WOMAC_Physical function,WOMAC_MHhysical function Grade,WOMAC_total,...,TA-G-100,Stance_CCI_TA-RF,Stance_CCI_G-MH,Stance_CCI_TA-G,Swing_CCI_TA-RF,Swing_CCI_G-MH,Swing_CCI_TA-G,swing/stance_cci(TA_RF),swing/stance_cci(G-MH),swing/stance_CCI(TA-G)
0,13485,Testing,O,13,Moderate,2,Mild,50,Severe,65,...,77.138703,20.102298,196.814698,20.195388,120.246529,81.908660,120.858352,5.404777,0.410371,5.606843
1,14068,Testing,O,9,Moderate,5,Mild,50,Severe,66,...,50.641608,29.739039,303.558550,27.946614,70.350635,339.850708,77.711109,2.435071,1.118654,2.875488
2,14076,Testing,O,9,Moderate,5,Mild,49,Severe,66,...,42.016959,136.708678,161.026615,125.546798,85.352214,75.971069,114.803019,0.607191,0.462269,0.909918
3,14443,Testing,O,14,Moderate,5,Mild,45,Severe,66,...,82.400223,98.003232,73.762646,85.101812,112.512842,55.009211,173.039607,1.136396,0.734806,1.916109
4,14444,Testing,O,14,Moderate,5,Mild,45,Severe,67,...,875.337685,245.194011,73.145649,478.922628,326.323460,14.577088,805.935009,1.333806,0.214086,1.725034
5,13242,Testing,O,10,Moderate,5,Mild,51,Severe,68,...,102.894813,95.250873,140.180174,100.070691,226.303541,27.287724,224.923047,2.378476,0.197126,2.272504
6,13339,Testing,O,14,Moderate,6,Mild,49,Severe,69,...,102.992800,126.079106,133.134121,125.079831,111.620435,111.078633,127.353750,0.863466,0.823439,1.016759
7,13355,Testing,O,13,Moderate,6,Mild,50,Severe,69,...,56.647518,289.678146,107.283677,136.757534,157.444790,74.011213,184.759008,0.537543,0.712175,1.357576
8,13524,Testing,O,13,Moderate,5,Mild,49,Severe,69,...,197.889782,81.484162,64.552967,160.619364,111.610211,24.071469,327.607628,1.353547,0.351191,2.063614
9,14137,Testing,O,12,Moderate,6,Mild,52,Severe,70,...,258.253671,227.427524,99.307493,253.893765,279.176459,32.862801,399.608138,1.197881,0.356753,1.550018


In [6]:
X = dataset.iloc[:,17:]
y = dataset.iloc[:, 9]

In [7]:
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
'''

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)\n'

In [ ]:
from sklearn.feature_selection import SelectKBest
bestfeatures = SelectKBest(k=10)
fit = bestfeatures.fit(X,y)


dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)



featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']


number_of_feat=int(input("Number Of Best Features: "))


imp=featureScores.nlargest(number_of_feat,'Score')
print(imp)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [21 22 23 24] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
list_of_feat=[]

for i in range(number_of_feat):
  list_of_feat.append(imp.iloc[:,0:1].values[i][0])

In [ ]:
X_new = dataset[list_of_feat]
y_new = dataset.iloc[:, 9]

In [ ]:
X_train=X_new.iloc[:120,:]  
X_test=X_new.iloc[120:,:]
y_train=y_new.iloc[:120]
y_test=y_new.iloc[120:]

#Model Result Analyzing Functions

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
import matplotlib.pyplot as plt 
import seaborn as sns


def R2_Score(temp_y,temp_pred):
  return r2_score(temp_y,temp_pred)


def MS_error(temp_y,temp_pred):
  return mean_squared_error(temp_y, temp_pred)




def observe_plot(temp_y,temp_pred,width,height):
  plt.figure(figsize=(width,height))
  plt.scatter(temp_y, temp_pred, color = "blue")
  plt.xlabel("Actual")
  plt.ylabel("Predicted")
  plt.show()


def pred_dataframe(temp_y,temp_pred):
  return pd.DataFrame(
        {
            "Actual Value":temp_y,
            "Predicted Value":temp_pred,
            "Difference":temp_y-temp_pred
        })




def reg_plot(pred_y_df,width,height):
  sns.set(rc={'figure.figsize':(width,height)})
  sns.regplot(x="Actual Value", y="Predicted Value", data=pred_y_df);





In [ ]:
!pip install shap
import shap

def explain_shap(model,temp_X,max_dis):
  explainer = shap.Explainer(model.predict, temp_X)
  shap_values = explainer (temp_X)
  shap.plots.bar(shap_values,max_display=max_dis)

#Linear Regression

###Model Training

In [ ]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
regr.fit(X_train, y_train)

###Training Results

In [ ]:
y_pred_train = regr.predict(X_train)
print("R2 score of Training: ", R2_Score(y_train,y_pred_train))
print("Mean squared error: ",MS_error(y_train, y_pred_train))


In [ ]:
#observe_plot(y_train,y_pred_train,width=15,height=10)

In [ ]:
pred_y_df = pred_dataframe(y_train,y_pred_train)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_train,20)



---



###Testing Results

In [ ]:
y_pred_test = regr.predict(X_test)
print("R2 score of Testing: ", R2_Score(y_test,y_pred_test))
print("Mean squared error: ",MS_error(y_test,y_pred_test))

In [ ]:
#observe_plot(y_test,y_pred_test,width=15,height=10)

In [ ]:
pred_y_df = pred_dataframe(y_test,y_pred_test)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_test,20)



---



###All Data

In [ ]:
y_pred_all = regr.predict(X_new)
print("R2 score of Testing: ", R2_Score(y_new,y_pred_all))
print("Mean squared error: ",MS_error(y_new,y_pred_all))

In [ ]:
pred_y_df = pred_dataframe(y_new,y_pred_all)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_new,20)



---



#Support Vector Regression (SVR) using linear and non-linear kernels

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

###Training Results

In [ ]:
y_pred_train = regr.predict(X_train)
print("R2 score of Training: ", R2_Score(y_train,y_pred_train))
print("Mean squared error: ",MS_error(y_train, y_pred_train))


In [ ]:
pred_y_df = pred_dataframe(y_train,y_pred_train)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_train,20)



---



###Testing Results

In [ ]:
y_pred_test = regr.predict(X_test)
print("R2 score of Testing: ", R2_Score(y_test,y_pred_test))
print("Mean squared error: ",MS_error(y_test,y_pred_test))

In [ ]:
pred_y_df = pred_dataframe(y_test,y_pred_test)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_test,20)



---



###All Data

In [ ]:
y_pred_all = regr.predict(X_new)
print("R2 score of Testing: ", R2_Score(y_new,y_pred_all))
print("Mean squared error: ",MS_error(y_new,y_pred_all))

In [ ]:
pred_y_df = pred_dataframe(y_new,y_pred_all)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_new,20)



---



#RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, y_train)


###Training Results

In [ ]:
y_pred_train = regr.predict(X_train)
print("R2 score of Training: ", R2_Score(y_train,y_pred_train))
print("Mean squared error: ",MS_error(y_train, y_pred_train))


In [ ]:
pred_y_df = pred_dataframe(y_train,y_pred_train)
reg_plot(pred_y_df,width=15,height=10)



---



###Testing Results

In [ ]:
y_pred_test = regr.predict(X_test)
print("R2 score of Testing: ", R2_Score(y_test,y_pred_test))
print("Mean squared error: ",MS_error(y_test,y_pred_test))

In [ ]:
pred_y_df = pred_dataframe(y_test,y_pred_test)
reg_plot(pred_y_df,width=15,height=10)



---



###All Data

In [ ]:
y_pred_all = regr.predict(X_new)
print("R2 score of Testing: ", R2_Score(y_new,y_pred_all))
print("Mean squared error: ",MS_error(y_new,y_pred_all))

In [ ]:
pred_y_df = pred_dataframe(y_new,y_pred_all)
reg_plot(pred_y_df,width=15,height=10)

#Ridge

[DOC](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [ ]:
from sklearn.linear_model import Ridge
regr = Ridge(alpha=1.0)
regr.fit(X_train, y_train)

###Training Results

In [ ]:
y_pred_train = regr.predict(X_train)
print("R2 score of Training: ", R2_Score(y_train,y_pred_train))
print("Mean squared error: ",MS_error(y_train, y_pred_train))


In [ ]:
pred_y_df = pred_dataframe(y_train,y_pred_train)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_train,20)



---



###Testing Results

In [ ]:
y_pred_test = regr.predict(X_test)
print("R2 score of Testing: ", R2_Score(y_test,y_pred_test))
print("Mean squared error: ",MS_error(y_test,y_pred_test))

In [ ]:
pred_y_df = pred_dataframe(y_test,y_pred_test)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_test,20)



---



###All Data

In [ ]:
y_pred_all = regr.predict(X_new)
print("R2 score of Testing: ", R2_Score(y_new,y_pred_all))
print("Mean squared error: ",MS_error(y_new,y_pred_all))

In [ ]:
pred_y_df = pred_dataframe(y_new,y_pred_all)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_new,20)



---



#Lasso

[DOC](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

In [ ]:
from sklearn import linear_model
regr = linear_model.Lasso(alpha=0.1)
regr.fit(X_train, y_train)


###Training Results

In [ ]:
y_pred_train = regr.predict(X_train)
print("R2 score of Training: ", R2_Score(y_train,y_pred_train))
print("Mean squared error: ",MS_error(y_train, y_pred_train))


In [ ]:
pred_y_df = pred_dataframe(y_train,y_pred_train)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_train,20)



---



###Testing Results

In [ ]:
y_pred_test = regr.predict(X_test)
print("R2 score of Testing: ", R2_Score(y_test,y_pred_test))
print("Mean squared error: ",MS_error(y_test,y_pred_test))

In [ ]:
pred_y_df = pred_dataframe(y_test,y_pred_test)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_test,20)



---



###All Data

In [ ]:
y_pred_all = regr.predict(X_new)
print("R2 score of Testing: ", R2_Score(y_new,y_pred_all))
print("Mean squared error: ",MS_error(y_new,y_pred_all))

In [ ]:
pred_y_df = pred_dataframe(y_new,y_pred_all)
reg_plot(pred_y_df,width=15,height=10)

In [ ]:
explain_shap(regr,X_new,20)